# Imports

In [ ]:
import numpy as np

from qiskit import QuantumCircuit
from LogicalQ.Logical import LogicalCircuit
from LogicalQ.Library.QECCs import steane_code
from LogicalQ.Library.HardwareModels import hardware_models_Quantinuum
from LogicalQ.Execution import execute_circuits

# Two Plaquette SU(2) Hamiltonian Model

In [ ]:
def TwoPlaquetteSU2TrotterStep(lqc, x, t):
    lqc.cx(1, 0)
        
    lqc.ry(-1/2 * x * t, 0)
    lqc.rz(-3/8 * t, 0)
    
    lqc.cx(1, 0)
    
    lqc.ry(-3/2 * x * t, 0)
    lqc.rz(-9/8 * t, 1)
    
    lqc.rz(-9/4 * t, 0)
    lqc.ry(-3 * x * t, 1)

    lqc.cx(0, 1)

    lqc.ry(-1 * x * t, 1)

    lqc.cx(0, 1)

    lqc.ry(-3/2 * x * t, 0)
    lqc.rz(-9/8 * t, 1)

    lqc.cx(1, 0)

    lqc.rz(-3/8 * t, 0)
    lqc.ry(-1/2 * x * t, 0)

    lqc.cx(1, 0)

def TwoPlaquetteSU2(x, dt, Nt, mitigation=False):
    lqc = LogicalCircuit(2, **steane_code)
    lqc.encode([0,1], initial_states=[0,0])

    for _ in range(Nt):
        TwoPlaquetteSU2TrotterStep(lqc, x, dt)
    lqc.measure_all()

    if mitigation:
        for _ in range(Nt):
            TwoPlaquetteSU2TrotterStep(lqc, x, dt)
        for _ in range(Nt):
            TwoPlaquetteSU2TrotterStep(lqc, x, -dt)
        
        lqc.measure_all()

    return lqc

# LogicalCircuit Simulations

In [ ]:
lqc_su2 = TwoPlaquetteSU2(2, 0.1, 10, mitigation=False)
lqc_su2.draw("mpl")

In [ ]:
result = execute_circuits(
    lqc_su2,
    backend="aer_simulator", method="statevector",
    hardware_model=hardware_models_Quantinuum["H2-2"], coupling_map=None,
    shots=int(1E3)
)

# Quantinuum Hardware Execution

In [ ]:
result = execute_circuits(
    lqc_su2,
    backend="quantinuum_H2-2E",
    shots=int(1E3)
)